In [1]:
import polars as pl 

In [11]:
tidbit = pl.read_csv('all_reviews.csv', n_rows = 100)

In [13]:
tidbit.head()

recommendationid,appid,game,author_steamid,author_num_games_owned,author_num_reviews,author_playtime_forever,author_playtime_last_two_weeks,author_playtime_at_review,author_last_played,language,review,timestamp_created,timestamp_updated,voted_up,votes_up,votes_funny,weighted_vote_score,comment_count,steam_purchase,received_for_free,written_during_early_access,hidden_in_steam_china,steam_china_location
i64,i64,str,i64,i64,i64,i64,i64,i64,i64,str,str,i64,i64,i64,i64,i64,f64,i64,i64,i64,i64,i64,str
148919893,10,"""Counter-Strike""",76561199036724879,0,3,197,197,197,1698336369,"""russian""","""старость""",1698336397,1698336397,1,0,0,0.0,0,1,0,0,1,""""""
148919350,10,"""Counter-Strike""",76561198826729322,0,21,441,37,441,1698335809,"""russian""","""Лучше кс 2""",1698335821,1698335821,1,0,0,0.0,0,1,0,0,1,""""""
148913051,10,"""Counter-Strike""",76561199101237625,7,1,1440,1440,1313,1698338635,"""turkish""","""çoh iyi ama pahalı""",1698329862,1698329862,1,0,0,0.0,0,1,0,0,1,""""""
148912714,10,"""Counter-Strike""",76561198347582422,11,4,1636,83,1612,1698341834,"""russian""","""топ""",1698329555,1698329555,1,0,0,0.0,0,1,0,0,1,""""""
148912575,10,"""Counter-Strike""",76561198363716821,0,2,197,41,197,1698329401,"""english""","""GOAT Game !!""",1698329419,1698329419,1,0,0,0.0,0,1,0,0,1,""""""


In [14]:
lf = pl.scan_csv('all_reviews.csv')

In [ ]:
# processing in batches 
transformed_lf = (
    lf
    .filter(pl.col('language') == 'english')
    .drop(['appid', 'hidden_in_steam_china', 'steam_china_location', 'timestamp_updated', 'received_for_free', 'written_during_early_access', 'language', 'author_last_played', 'recommendationid'])
    .with_columns((pl.col('timestamp_created')*1000).cast(pl.Datetime('ms')))
)

In [16]:
cleaned_df = transformed_lf.collect(streaming=True)

In [ ]:
# getting rid of any characters that aren't english. could be bad for any sentiment analysis packages 
cleaned_df = cleaned_df.filter(cleaned_df['review'].str.contains(r"^[\x00-\x7F]+$", literal=False))

In [17]:
cleaned_df

game,author_steamid,author_num_games_owned,author_num_reviews,author_playtime_forever,author_playtime_last_two_weeks,author_playtime_at_review,review,timestamp_created,voted_up,votes_up,votes_funny,weighted_vote_score,comment_count,steam_purchase
str,i64,i64,i64,i64,i64,i64,str,datetime[ms],i64,i64,i64,f64,i64,i64
"""Counter-Strike""",76561198363716821,0,2,197,41,197,"""GOAT Game !!""",2023-10-26 14:10:19,1,0,0,0.0,0,1
"""Counter-Strike""",76561198134752176,69,1,12107,0,12107,"""First crush. Always in my hear…",2023-10-26 07:41:36,1,0,0,0.0,0,0
"""Counter-Strike""",76561197985437504,0,5,42519,3,42515,"""best version of cs ever, histo…",2023-10-26 07:32:49,1,0,0,0.0,0,0
"""Counter-Strike""",76561199015747395,71,34,30,0,30,"""a game everyone needs, but not…",2023-10-25 18:11:55,1,0,0,0.0,0,1
"""Counter-Strike""",76561199479067349,5,6,209,200,209,"""old but gold""",2023-10-25 17:03:14,1,1,0,0.52381,0,1
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""Siluman Fantasy""",76561198010914579,3130,270,30,0,30,"""Let's start off with, I would …",2019-04-21 12:24:21,0,77,1,0.810085,0,0
"""Siluman Fantasy""",76561197988663774,3463,347,645,0,645,"""(Disclaimer: I know one of the…",2019-04-19 13:52:31,1,26,1,0.655006,0,0
"""Monster Girl Sketch Vol.01""",76561197988663774,3463,347,0,0,0,"""Sketchbook, not actually R18. …",2019-04-19 13:54:13,1,3,0,0.542915,0,0


In [18]:
cleaned_df.write_parquet('english_only.parquet')